In [1]:
%matplotlib inline
%config Completer.use_jedi = False # to use autocomplete
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import os
import json
import pandas as pd
import sklearn
from random import seed
from sklearn.model_selection import KFold
from sklearn import linear_model, ensemble, neighbors, tree
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
# from sklearn.preprocessing import PolynomialFeatures
# from texttable import Texttable
import scipy.optimize as optimization
RANDOM_SEED=0
seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
print(sklearn.__version__)
from itertools import combinations
from numpy.random import default_rng
import itertools
plt.rcdefaults()
# plt.rcParams.update({'font.family':'fantasy'})

1.2.1


In [2]:
def path_leaf(path): # get filename(with extension) from path
    head, tail = os.path.split(path)
    return tail or os.path.basename(head)


def file_in_folder(folder_path, flag=1): # get files in given folder, return list of filepath and filename
    file_list = []
    file_name = []
    for(dirpath, dirnames, filenames) in os.walk(folder_path):
#         print(filenames)
        
        for i in filenames:
            try:
                file_list += [dirpath + os.sep + i]
                file_name += [i]
            except:
                continue
        if flag == 0:
            break
        file_list.sort(key=path_leaf)
        file_name.sort()
    return [file_list, file_name]

In [5]:
data_folder = '/projects/p31434/lottery/replication/data/PPP_normalized_44'
file_list, file_name = file_in_folder(data_folder)
file_list = [i for i in file_list if '.csv' in i and '_30countries' in i]
file_name = [i for i in file_name if '.csv' in i and '_30countries' in i]
num_name_dic = {i: name for i, name in enumerate(file_name)}
name_num_dic = {name: i for i, name in enumerate(file_name)}

In [6]:
show_num_name_dic = {'num': [], 'name': [], 'author': [], 'gain_only': []}
for key, val in num_name_dic.items():
    show_num_name_dic['num'].append(key)
    show_num_name_dic['name'].append(val)
    if '_30countries' in val:
        show_num_name_dic['author'].append('l\'Haridon')
    elif 'rationality on the rise' in val:
        show_num_name_dic['author'].append('fehr')
    elif 'ch03_data.csv' == val or 'ch06_data.csv' == val:
        show_num_name_dic['author'].append('Bruhin')
    elif 'experiments on compound risk in' in val:
        show_num_name_dic['author'].append('abdellaoui')
    elif 'decisions with compound lotteries' in val:
        show_num_name_dic['author'].append('fan')
    elif 'accommodating stake effects under prospect' in val:
        show_num_name_dic['author'].append('bouchouicha')
    elif 'impatience and uncertainty' in val:
        show_num_name_dic['author'].append('Sutter')
    elif 'Monetary incentives in the loss domain' in val:
        show_num_name_dic['author'].append('Etchart')
    elif 'incentive effects on rist attitude' in val:
        show_num_name_dic['author'].append('Lefebvre')
    elif 'ellsberg revisited' in val:
        show_num_name_dic['author'].append('Halevy')
    elif 'on the interaction of risk and time' in val:
        show_num_name_dic['author'].append('Anderhub')
    elif 'how do risk attitudes affect measured' in val:
        show_num_name_dic['author'].append('Murad')
    elif 'the empirical relationship between' in val:
        show_num_name_dic['author'].append('Dean')
    elif 'on the empirical validity of cumulative' in val:
        show_num_name_dic['author'].append('Bernheim')
    else:
        print(val)
    
    df = pd.read_csv(file_list[key])
    if any(df['z1'] <= 0):
        show_num_name_dic['gain_only'].append('N')
    else:
        show_num_name_dic['gain_only'].append('Y')
num_name_df = pd.DataFrame.from_dict(show_num_name_dic)
print(num_name_df.to_latex(index=False))
num_name_df


\begin{tabular}{rlll}
\toprule
num & name & author & gain_only \\
\midrule
0 & Australia_data_30countries.csv & l'Haridon & N \\
1 & Belgium_data_30countries.csv & l'Haridon & N \\
2 & Brazil_data_30countries.csv & l'Haridon & N \\
3 & Cambodia_data_30countries.csv & l'Haridon & N \\
4 & Chile_data_30countries.csv & l'Haridon & N \\
5 & China_data_30countries.csv & l'Haridon & N \\
6 & Colombia_data_30countries.csv & l'Haridon & N \\
7 & Costa Rica_data_30countries.csv & l'Haridon & N \\
8 & Czech Republic_data_30countries.csv & l'Haridon & N \\
9 & Ethiopia_data_30countries.csv & l'Haridon & N \\
10 & France_data_30countries.csv & l'Haridon & N \\
11 & Germany_data_30countries.csv & l'Haridon & N \\
12 & Guatemala_data_30countries.csv & l'Haridon & N \\
13 & India_data_30countries.csv & l'Haridon & N \\
14 & Japan_data_30countries.csv & l'Haridon & N \\
15 & Kyrgyzstan_data_30countries.csv & l'Haridon & N \\
16 & Malaysia_data_30countries.csv & l'Haridon & N \\
17 & Nicaragua_data_30c

,num,name,author,gain_only
0,0,Australia_data_30countries.csv,l'Haridon,N
1,1,Belgium_data_30countries.csv,l'Haridon,N
2,2,Brazil_data_30countries.csv,l'Haridon,N
3,3,Cambodia_data_30countries.csv,l'Haridon,N
4,4,Chile_data_30countries.csv,l'Haridon,N
5,5,China_data_30countries.csv,l'Haridon,N
6,6,Colombia_data_30countries.csv,l'Haridon,N
7,7,Costa Rica_data_30countries.csv,l'Haridon,N
8,8,Czech Republic_data_30countries.csv,l'Haridon,N
9,9,Ethiopia_data_30countries.csv,l'Haridon,N


In [7]:
df = pd.read_csv('/projects/p31434/lottery/replication/data/PPP_normalized_pooled_data/30countries.csv')

In [10]:
a, b = np.unique(df['lottery'], return_counts=True)

In [13]:
useful_lottery_num = []
for i in range(b.shape[0]):
    if b[i] > 2900:
        useful_lottery_num.append(a[i])
print(useful_lottery_num, len(useful_lottery_num))

[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 23, 24, 25, 26, 27, 28, 30, 31, 32, 33, 34, 44] 24
